# Baby Boomers Thru Time
## Demographics Report by &lt; Katsiaryna Aliashkevich kia236 &gt;



### Creating Interactive Charts to Visualize Population Shifts over Time with Altair

Baby boomers (often shortened to boomers) are the demographic cohort following the Silent Generation and preceding Generation X. The generation is generally defined as people born from 1946 to 1964, during the post–World War II baby boom. The term is also used outside the United States but the dates, the demographic context and the cultural identifiers may vary. The baby boom has been described variously as a "shockwave"and as "the pig in the python." Baby boomers are often parents of late Gen Xers and Millennials. [from wikipedia](https://en.wikipedia.org/wiki/Baby_boomers).

In [ ]:
# Import the necessary libraries and data
import altair as alt
import pandas as pd
from vega_datasets import data

df_population = data.population()

In [ ]:
df_population.head()

,year,age,sex,people
0,1850,0,1,1483789
1,1850,0,2,1450376
2,1850,5,1,1411067
3,1850,5,2,1359668
4,1850,10,1,1260099


## Add in the "Boomer" label

As we can see from inspecting the dataframe, our data only gives us information for: 
  - year
  - age
  - sex
  - people
  
But, we want to be able to highlight just the people born between 1946 - 1964 as a separate group.  To accomplish this, we want to create a new categorical attribute - `Generation`

In [ ]:
values = []

for index, row in df_population.iterrows():
    if 1946 < (row['year'] - row['age']) < 1964:
      values.append("Baby Boomer")
    else:
      values.append("Other")

df_population['Generation'] = values

## Change the encoding for `sex`

The sex for "Male" is encoded as the number `1` and the sex for Female is encoded as `2`.

In [ ]:
sex = df_population['sex']
df_population['sex'] = sex.map({1: 'Male', 2: 'Female'})


In [ ]:
df_population.head()

,year,age,sex,people,Generation
0,1850,0,Male,1483789,Other
1,1850,0,Female,1450376,Other
2,1850,5,Male,1411067,Other
3,1850,5,Female,1359668,Other
4,1850,10,Male,1260099,Other


## Juxtapose Bar Charts Horizontally 

Create a bar chart of the population distribution in the year 1960, and horizontally juxtapose the bar chart of the population distribution for the year 1990. Plot the total number of people (ignoring the `sex` attribute).

Fix the y axis so it is equal in both plots.

In [ ]:

six = df_population.query("year == 1960")
nine = df_population.loc[df_population['year'] == 1990]
six.head(50)
purple = "#7D3C98"
gold = "#F4D03F"
purple_gold = alt.Scale(domain=('Baby Boomer', 'Other'),
                      range=[purple, gold])

chart1 = alt.Chart(six).mark_bar().encode(
    x=alt.X('age:N', axis=alt.Axis(title='Age')),
    y=alt.Y('sum(people)', scale=alt.Scale(domain=[0, 24000000]), axis=alt.Axis(title='Number of People')),
    color=alt.Color('Generation', scale=purple_gold),
).properties(
    title='Distribution of Ages in 1960'
)

chart2 = alt.Chart(nine).mark_bar().encode(
    x=alt.X('age:N', axis=alt.Axis(title='Age')),
    y=alt.Y('sum(people)', scale=alt.Scale(domain=[0, 24000000]), axis=alt.Axis(title='Number of People')),
    color=alt.Color('Generation', scale=purple_gold),
).properties(
    title='Distribution of Ages in 1990'
)

chart1 | chart2

alt.HConcatChart(...)

## Show the Population Change Over Time with a Slider

Start the slider at 1900.

In [ ]:
slider = alt.binding_range(min=1900, max=2000, step=10, name='Select Year:')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2000})
purple = "#7D3C98"
gold = "#F4D03F"
purple_gold = alt.Scale(domain=('Baby Boomer', 'Other'),
                      range=[purple, gold])

alt.Chart(df_population).mark_bar().encode(
    x=alt.X('age:O', title='Age'),
    y=alt.Y('sum(people)', scale=alt.Scale(domain=(0, 24000000)), axis=alt.Axis(title='Number of People')),
    color=alt.Color('Generation', scale=purple_gold)
).properties(
    title= 'Population Distribution by Age in the USA',
    width= 700
).add_selection(
    select_year
).transform_filter(
    select_year
).configure_facet(
    spacing=8
)

alt.Chart(...)

## Linking

Let's take a closer look at just the year 2000 data, and find what the distribution of sex is for each individual age grouping.  Plot the distribution of ages as a bar chart for just the year 2000, and link a histogram that will plot the distribution of sex for the current selection. When a bar on the top chart is selected, indicate its selection by turning the other bars light gray.  The histogram of the sex distribution below it should be a horizontal bar chart. 


In [ ]:
purple = "#7D3C98"
gold = "#F4D03F"
purple_gold = alt.Scale(domain=('Baby Boomer', 'Other'),
                      range=[purple, gold])
pts = alt.selection(type="single", encodings=['x'])

two = df_population.loc[df_population['year'] == 2000]
chart1 = alt.Chart(two).mark_bar().encode(
    x=alt.X('age:N', axis=alt.Axis(title='Age')),
    y=alt.Y('sum(people)', scale=alt.Scale(domain=[0, 24000000]), axis=alt.Axis(title='Number of People')),
    color=alt.condition(pts,
                     alt.Color("Generation:N", scale=purple_gold),
                     alt.value("#bababa"))
).properties(
    title='Distribution of Ages in 2000',
    width = 700
).add_selection(pts)

chart2 = alt.Chart(two).mark_bar().encode(
    x=alt.X('people:Q', scale=alt.Scale(), axis=alt.Axis(title='Number of People')),
    y=alt.Y('sex:N', axis=alt.Axis(title='Sex')),
    color=alt.Color('Generation', scale=purple_gold),order=alt.Order(
      'Generation',
      sort='descending'
    )
).properties(
    title='Distribution of Sex for Above Age Selection',
    width = 700
).transform_filter(
    pts
)

alt.vconcat(
    chart1 & chart2
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)

## Combine into One Chart

Let's visualize all the data by incorporating the year selection slider so that you can select which year of data you are viewing. Retain the ability to just select one age group for the sex distribution, and default to no age group selected.

Add a tooltip so you can see exactly how many people are in the age range for the top "Distribution of Ages for the Selected Year" histogram. 

In [ ]:
purple = "#7D3C98"
gold = "#F4D03F"
purple_gold = alt.Scale(domain=('Baby Boomer', 'Other'),
                      range=[purple, gold])

pts = alt.selection(type="single", encodings=['x'])

slider = alt.binding_range(min=1900, max=2000, step=10, name='Select Year:')
select_year = alt.selection_single(name="year", fields=['year'],
                                   bind=slider, init={'year': 2000})
chart1 = alt.Chart(df_population).mark_bar().encode(
    x=alt.X('age:N', axis=alt.Axis(title='Age')),
    y=alt.Y('sum(people)', scale=alt.Scale(domain=[0, 24000000]), axis=alt.Axis(title='Number of People')),
    color=alt.condition(pts,
                     alt.Color("Generation:N", scale=purple_gold),
                     alt.value("#bababa")),
    tooltip=['age', 'people']
).interactive(
).properties(
    title= 'Distribution of Ages in Selected Year',
    width= 700
).add_selection(
    select_year
).transform_filter(
    select_year
).add_selection(pts)

chart2 = alt.Chart(df_population).mark_bar().encode(
    x=alt.X('people:Q', scale=alt.Scale(), axis=alt.Axis(title='Number of People')),
    y=alt.Y('sex:N', axis=alt.Axis(title='Sex')),
    color=alt.Color('Generation:N', scale=purple_gold),order=alt.Order(
      'Generation',
      sort='descending'
    )
).properties(
    title='Distribution of Sex for Above Age Selection',
    width = 700
).transform_filter(
    pts
).transform_filter(
    select_year
)

alt.vconcat(
    chart1 & chart2
).resolve_legend(
    color="independent",
    size="independent"
)

alt.VConcatChart(...)